In [1]:
!pip install transformers peft datasets accelerate -U

In [29]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Direct to your assignment folder.
%cd /content/drive/MyDrive/project-m2-2024-jim

/content/drive/MyDrive/project-m2-2024-jim


In [32]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='data/mcq_aquarat_dataset.jsonl')

# Print dataset information
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['subject', 'question', 'answer'],
        num_rows: 97467
    })
})


In [40]:
from datasets import load_dataset


# Shuffle the entire training dataset
shuffled_dataset = dataset['train'].shuffle(seed=42)

# Define the size for the subsets (e.g., 10% for training, 10% for validation)
subset_size = len(dataset['train']) // 80

# Generate the training subset (first 10%)
train_subset = shuffled_dataset.select(range(0, subset_size))

# Generate the validation subset (next 10%)
validation_subset = shuffled_dataset.select(range(subset_size, 2 * subset_size))

# Print the subset information
print("Training Subset:")
print(train_subset)

print("\nValidation Subset:")
print(validation_subset)

#### Having a small training dataset and validation dataset


Training Subset:
Dataset({
    features: ['subject', 'question', 'answer'],
    num_rows: 1218
})

Validation Subset:
Dataset({
    features: ['subject', 'question', 'answer'],
    num_rows: 1218
})


In [35]:
#### ignore this

from transformers import AutoModelForCausalLM, AutoTokenizer

model_output_dir = "./model_output_second"
#model = AutoModelForCausalLM.from_pretrained(model_output_dir)
#model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data_1(example):
    inputs = tokenizer(example["question"], padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    labels = tokenizer(example["answer"], padding="max_length", truncation=True, max_length=1024, return_tensors="pt").input_ids
    labels[labels == tokenizer.pad_token_id] = -100  # Set padding tokens to -100 to ignore them in the loss
    return {"input_ids": inputs.input_ids.squeeze(), "attention_mask": inputs.attention_mask.squeeze(), "labels": labels.squeeze()}


def preprocess_data(example):
    # Tokenize and pad the question
    inputs = tokenizer(example["question"], padding="max_length", truncation=True, max_length=1024, return_tensors="pt")

    # Convert the single letter answer to an integer index
    label_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}  # Assuming the choices are A, B, C, and D
    label = label_map[example["answer"].strip()]  # Convert label to integer

    # Prepare the inputs and labels
    return {
        "input_ids": inputs.input_ids.squeeze(),
        "attention_mask": inputs.attention_mask.squeeze(),
        "labels": torch.tensor(label)
    }
# Apply preprocessing to the train subset
train_dataset = train_subset.map(preprocess_data_1, remove_columns=["subject", "question", "answer"])

validation_dataset = validation_subset.map(preprocess_data_1, remove_columns=["subject", "question", "answer"])



Map:   0%|          | 0/1218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1218 [00:00<?, ? examples/s]

In [12]:
#### ignore this

import torch
from torch.utils.data import Dataset, DataLoader
#from transformers import BertTokenizer, BertForMultipleChoice
from transformers import AutoModelForCausalLM, AutoTokenizer

# Custom Dataset class
class MCQADataset(Dataset):
    def __init__(self, data):
        self.questions = data['question']
        self.answers = data['answer']
        self.answer_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}  # Example answer mapping

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]
        answer_idx = self.answer_map[answer]  # Convert letter answer to index
        return question, answer_idx


# Create dataset and dataloader
data = MCQADataset(train_subset)
dataloader = DataLoader(data, batch_size=2, shuffle=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertForMultipleChoice.from_pretrained('bert-base-uncased')
model_output_dir = "./model_output_second"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)


tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token


model.to(device)  # Move model to the device
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()

for questions, answers in dataloader:
    # Tokenize questions
    input_tensors = []
    attention_masks = []
    for q in questions:
        inputs = tokenizer(q, return_tensors="pt", padding="max_length", truncation=True, max_length=1024)
        input_tensors.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    # Stack inputs into a single tensor (batch_size, num_choices, sequence_length)
    input_tensors = torch.stack(input_tensors).to(device)  # Move to device
    attention_masks = torch.stack(attention_masks).to(device)  # Move to device

    # Ensure answers are in tensor form and move to device
    answer_tensors = torch.tensor(answers).to(device)
    print(answer_tensors.shape)

    # Forward pass
    outputs = model(input_ids=input_tensors, attention_mask=attention_masks)
    print(outputs.logits.shape)

    # Compute loss
    loss = loss_function(outputs.logits, answer_tensors)

    # Backpropagation and optimization steps
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



<ipython-input-12-8ac591633e88>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  answer_tensors = torch.tensor(answers).to(device)


torch.Size([2])
torch.Size([2, 1, 1024, 50257])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of size: : [2]

In [24]:
#### ignore this

import re

class MCQModel:
    def __init__(self, generator_model):
        self.generator = generator_model

    def generate_reasoning(self, question_with_options):
        """
        This method uses the generator model to produce detailed reasoning for the MCQ.
        """
        # Implement the logic to generate reasoning using the generator model.
        # Here, we assume the generator model has a method `generate` that takes a prompt and returns a reasoning.
        prompt = question_with_options + "\nAnswer with reasoning:"
        reasoning = self.generator.generate(prompt)
        return reasoning

    def extract_answer(self, reasoning):
        """
        This post-processing function extracts the single letter answer from the generated reasoning.
        """
        # Implement logic to extract the final answer option from the reasoning.
        # For simplicity, let's assume the answer is mentioned at the end of the reasoning.
        match = re.search(r'Answer:\s*([A-E])', reasoning)
        if match:
            return match.group(1)
        else:
            # Fallback to first letter detected if specific pattern is not found.
            for option in ['A', 'B', 'C', 'D', 'E']:
                if option in reasoning:
                    return option
        return None

    def prediction_step_mcqa(self, question, options):
        """
        This function performs the prediction step for MCQ answering.
        """
        question_with_options = f"Question: {question}\n\nOptions:\n{options}\nAnswer:"
        reasoning = self.generate_reasoning(question_with_options)
        answer = self.extract_answer(reasoning)
        return answer



model_output_dir = "./model_output_second"
#model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model = AutoModelForCausalLM.from_pretrained("gpt2")

#mcq_model = MCQModel(generator_model)

def generate_response(prompt, model, tokenizer, max_length=50):
    # Tokenize the input prompt with attention mask
    inputs = tokenizer.encode(prompt, return_tensors='pt', padding=True, truncation=True)
    attention_mask = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)['attention_mask']

    # Generate response from GPT-2
    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Explicitly set pad token ID to EOS token ID
    )

    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

#question = "Question: what is the sum of the greatest common factor and the lowest common multiple of 36 and 56 ?"
#options = "A:22\nB:24\nC:26\nD:28\nE:30"
#prompt_text = question
#answer = model(question + options)

# Prompt to send to GPT-2
prompt = "What is the capital of France? A) Paris; B) Rome; C) London; D) Madrid."

# Get the response from GPT-2
response = generate_response(prompt, model, tokenizer)

# Print the response
print(response)


What is the capital of France? A) Paris; B) Rome; C) London; D) Madrid.

The capital is Paris, and the city is called the "Parisian capital." The capital was founded in 1789, when


In [14]:
#### ignore this

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

class MCQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.questions = data['question']
        self.answers = data['answer']
        self.answer_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}  # Example answer mapping
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]
        answer_idx = self.answer_map[answer]  # Convert letter answer to index

        # Parse the question and options
        question_text, options_text = question.split("\n\nOptions:\n")
        choices = options_text.split("\n")[1:]  # Split the options by newline and skip the first line

        # Tokenize the input text for each choice
        input_ids = [self.tokenizer(f"{question_text} {choice}", return_tensors="pt", padding='max_length', max_length=512, truncation=True).input_ids.squeeze(0) for choice in choices]

        # Stack input_ids
        input_ids = torch.stack(input_ids, dim=0)

        return input_ids, answer_idx



# Tokenizer and dataset
model_output_dir = "./model_output_second"
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
tokenizer.pad_token = tokenizer.eos_token

dataset = MCQADataset(train_subset, tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # Adjust batch_size as needed

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GPT-2 model fine-tuned for sequence classification
#model = GPT2ForSequenceClassification.from_pretrained('path_to_your_fine_tuned_gpt2')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()

for input_ids_batch, answers in dataloader:
    # Move input tensors and target tensors to device
    input_ids_batch = input_ids_batch.to(device)  # Shape: (batch_size, num_choices, sequence_length)
    answers = answers.to(device)  # Shape: (batch_size)

    # Reshape input_ids_batch to (batch_size * num_choices, sequence_length)
    input_ids_batch = input_ids_batch.view(-1, input_ids_batch.size(-1))

    expanded_answers = answers.repeat_interleave(5)  # Each question has 5 choices

    # Forward pass
    outputs = model(input_ids=input_ids_batch, labels=expanded_answers)


    # The model's outputs include loss if labels are provided
    loss = outputs.loss

    # Backpropagation and optimization steps
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Loss: {loss.item()}")


ValueError: Expected input batch_size (2555) to match target batch_size (4).

In [36]:
#### ignore this

import torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['c_attn', 'c_proj', 'mlp.c_fc', 'mlp.c_proj']  # Targeting GPT-2 specific modules
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
model = get_peft_model(model, peft_config)

# Training arguments for the first fine-tuning
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=10,
    output_dir="./model_output_mcqa_1",
    optim="adamw_torch",
)

# Trainer setup for the first fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

# Train the model on the first dataset
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss


KeyboardInterrupt: 

In [65]:
#### Continue here
#### The idea is to customize a loss function for the MCQA data training:
#### If we just send the prompt (MCQ) to the model,
#### we assume it's able to generate an answer, but not in the right format
#### for example, with the MCQ "What is the biggest? A)2; B)4; C)6; D)8"
#### we assume the model generates an answer like "8 is the biggest"
#### but for MCQA we hope it generates "D"
#### so we apply a post-process function that compares the genrated answer like "8 is the biggest" with all the options (2, 4, 6, 8)
#### and choose the closest option (8) and take that choice (D) as the "final output"
#### so the new loss function would be to compare the label/target (D) and the "final output"

import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch


# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Tokenize the data
def tokenize_function(example):
    input_text = example["question"]
    target_text = example["answer"]

    # Extracting choices
    choices = [line.split(":")[1] for line in input_text.split("\n") if line.startswith(('A:', 'B:', 'C:', 'D:', 'E:'))]
    choice_labels = [line.split(":")[0] for line in input_text.split("\n") if line.startswith(('A:', 'B:', 'C:', 'D:', 'E:'))]

    inputs = tokenizer(input_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    targets = tokenizer(target_text, padding="max_length", truncation=True, max_length=4, return_tensors="pt")

    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "labels": targets["input_ids"].squeeze(),
        "choices": choices,
        "choice_labels": choice_labels
    }

# Tokenize the dataset
tokenized_datasets = [tokenize_function(item) for item in train_subset]

# Custom dataset class
class MCQADataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_datasets):
        self.data = tokenized_datasets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create the dataset
train_dataset = MCQADataset(tokenized_datasets)

# Inspect the first sample
sample = train_dataset[0]
print(sample)


{'input_ids': tensor([24361,    25,  1318,   389,   374,  2266,  2613,    11,   275,  4171,
         2613,   290,   266,  2330,  2613,   287,   257,  6131,    13,  1867,
          318,   262,  8064,   286,   262,  1271,   286,  4171,  2613,   284,
          262,  2472,   645,    13,   286,  2613,   287,  2846,   286,   374,
           11,   275,   290,   266, 40791,   198,   198, 29046,    25,   198,
           32,    25,    81,  1220,   357,    81,  1343,   275,  1343,   266,
            8,   198,    33,    25,    81,  1635,   357,    81,  1343,   275,
         1343,   266,     8,   198,    34, 37498,    81,  1343,   275,  1343,
          266,     8,   198,    35,    25,    81,  1220,   357,    81,  1343,
          275,  1267,   198,    36,    25,    81,  1220,   357,    65,  1343,
          266,     8,   198, 33706,    25, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

In [82]:
#### this collator was meant to ensure that columns "choices" and "choice labels" are kept
from transformers import DataCollatorWithPadding

class DataCollatorForMCQA(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        batch["choices"] = [feature["choices"] for feature in features]
        batch["choice_labels"] = [feature["choice_labels"] for feature in features]
        return batch
data_collator = DataCollatorForMCQA(tokenizer)

In [66]:
#### we might also need a better way to find the best match
def post_process_answer(generated_answer, choices):
    best_choice = None
    best_score = float('inf')
    for choice in choices:
        score = levenshtein_distance(generated_answer, choice)
        if score < best_score:
            best_score = score
            best_choice = choice
    return best_choice

def levenshtein_distance(a, b):
    if len(a) < len(b):
        return levenshtein_distance(b, a)

    if len(b) == 0:
        return len(a)

    previous_row = range(len(b) + 1)
    for i, c1 in enumerate(a):
        current_row = [i + 1]
        for j, c2 in enumerate(b):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


In [74]:
# Custom loss function
def custom_loss_function(logits, labels, choices, choice_labels):
    # Generate the most probable sequence
    generated_ids = torch.argmax(logits, dim=-1)
    generated_answers = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # Post-process the generated answers
    post_processed_answers = [post_process_answer(answer, choice) for answer, choice in zip(generated_answers, choices)]

    # Convert post-processed answers to choice labels
    post_processed_choice_labels = [choice_labels[choices.index(answer)] for answer, choices in zip(post_processed_answers, choices)]

    # Calculate the loss with respect to the correct choice
    correct_choice_labels = [choice_labels.index(labels[i]) for i in range(len(labels))]
    post_processed_indices = [choice_labels.index(label) for label in post_processed_choice_labels]

    loss = F.cross_entropy(torch.tensor(post_processed_indices), torch.tensor(correct_choice_labels))

    return loss


In [81]:
#### now the problem is, as shown by the print of "inputs", it doesn't contain "labels"
# Custom Trainer class
class CustomTrainer(Trainer):
    print(inputs)
    def compute_loss(self, model, inputs, return_outputs=False):
        print(inputs)
        labels = inputs.pop("labels")
        choices = inputs.pop("choices")
        choice_labels = inputs.pop("choice_labels")

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # Custom loss computation
        loss = custom_loss_function(logits, labels, choices, choice_labels)

        return (loss, outputs) if return_outputs else loss


tensor([[24361,    25,   644,   318,   262,  2160,   286,   262,  6000,  2219,
          5766,   290,   262,  9016,  2219,  3294,   286,  4570,   290,  7265,
          5633]])


In [84]:
#### as long as the function "compute_loss" works, which means we can access columns "labels" in the train_dataset
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)
print(train_dataset[0])
# Initialize the custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'input_ids': tensor([24361,    25,  1318,   389,   374,  2266,  2613,    11,   275,  4171,
         2613,   290,   266,  2330,  2613,   287,   257,  6131,    13,  1867,
          318,   262,  8064,   286,   262,  1271,   286,  4171,  2613,   284,
          262,  2472,   645,    13,   286,  2613,   287,  2846,   286,   374,
           11,   275,   290,   266, 40791,   198,   198, 29046,    25,   198,
           32,    25,    81,  1220,   357,    81,  1343,   275,  1343,   266,
            8,   198,    33,    25,    81,  1635,   357,    81,  1343,   275,
         1343,   266,     8,   198,    34, 37498,    81,  1343,   275,  1343,
          266,     8,   198,    35,    25,    81,  1220,   357,    81,  1343,
          275,  1267,   198,    36,    25,    81,  1220,   357,    65,  1343,
          266,     8,   198, 33706,    25, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

KeyError: 'choices'